Preparing to submit wold stranded samples....

This got a little messy because a couple libraries needed a top up.

In [1]:
import os
import sys
import requests
import pandas
import paramiko
import re
import json
from IPython import display
from pathlib import Path
import configparser

In [2]:
from curation_common import *
from htsworkflow.submission.encoded import DCCValidator

In [3]:
from htsworkflow.submission.encoded import Document
from htsworkflow.submission.aws_submission import run_aws_cp
from htsworkflow.util.api import (
    add_auth_options,
    make_auth_from_opts,
    HtswApi,
)

In [4]:
ls /etc/ssl/certs/ca-certificates.crt

/etc/ssl/certs/ca-certificates.crt


In [5]:
import urllib.request
import ssl

ctx = ssl.SSLContext()
ctx.verify_mode = ssl.CERT_NONE
url = "https://jumpgate.caltech.edu/library"
urllib.request.urlopen(url, context=ctx)

In [6]:
config = configparser.ConfigParser()
config.read([os.path.expanduser('~/.htsworkflow.ini'),
             '/etc/htsworkflow.ini'
             ])

SECTION = 'sequence_archive'
if config.has_section(SECTION):
    apiid = config.get(SECTION, 'apiid')
    apikey = config.get(SECTION, 'apikey')
    apihost = config.get(SECTION, 'host')

auth = {'apiid': apiid, 'apikey': apikey }
htsw = HtswApi(apihost, auth)

In [7]:
# live server & control file
server = ENCODED('www.encodeproject.org')
spreadsheet_name = Path('~/woldlab/ENCODE/stranded-24038-24061-rush-resubmit.xlsx').expanduser()
engine=None
#engine='odf'

# test server & datafile
#server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-encode3-limb-2017-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [8]:
award = 'UM1HG009443'

# Confirm biosample donor is right

In [9]:
print(spreadsheet_name)
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)

for i, row in biosample.iterrows():
    library_id = row["library_id:skip"]
    cdna = row["cDNA_sample:skip"]
    description = row["description"]
    donor = row["donor"]
    match = re.search("Rush ID_(?P<rushid>[0-9]+)", description)
    assert match.group("rushid") == donor[len("john-stamatoyannopoulos:E"):], "{} {} {}".format(library_id, match.group("rushid"), donor)
    match = re.search("(?P<encid>ENC4_cDNA([0-9]+))", description)
    assert match.group("encid") == cdna, "{} {} {}".format(library_id, match.group("encid"), cdna)
    

/home/diane/woldlab/ENCODE/stranded-24038-24061-rush-resubmit.xlsx


# Confirm experiment is right

In [10]:
print(spreadsheet_name)
experiment = pandas.read_excel(spreadsheet_name, sheet_name='Experiment', header=0, engine=engine)

for i, row in experiment.iterrows():
    library_id = row["library_id:skip"]
    library_info = htsw.get_library(library_id)
    assert library_info["library_name"] == row["description"], "{} {} != {}".format(library_id, library_info["library_name"], row["description"])

/home/diane/woldlab/ENCODE/stranded-24038-24061-rush-resubmit.xlsx


# Lookup biosample ontologies

Lookup any biosample ontologies that are already present

In [11]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)

for i, row in biosample.iterrows():
    if not pandas.isnull(row.accession) and row.accession.startswith('E'):
        obj = server.get_json(row.accession)
        biosample_ontology = obj['biosample_ontology']
        if isinstance(biosample_ontology, dict):
            biosample.loc[i, 'biosample_ontology'] = biosample_ontology['@id']
            biosample.loc[i, 'biosample_term_name:skip'] = biosample_ontology['term_name']
        biosample.loc[i, 'source'] = obj['source']['@id']
            
biosample

,uuid,accession,rush_id:skip,age:skip,age_units:skip,sex:skip,clinical_status:skip,library_id:skip,cDNA_sample:skip,description,biosample_ontology,biosample_term_name:skip,aliases:array,organism,source,donor,lab,award
0,NaN,NaN,E3453103,90 or above,year,female,NCI,24038,ENC4_cDNA649,human_brain_Rush ID_3453103_BS49_ENC4_cDNA649,/biosample-types/tissue_UBERON_0006483/,mid frontal cortex,barbara-wold:ENC4_cDNA649,/organisms/human/,/sources/rush-university/,john-stamatoyannopoulos:E3453103,barbara-wold,UM1HG009443
1,NaN,NaN,E4163791,90 or above,year,female,NCI,24039,ENC4_cDNA650,human_brain_Rush ID_4163791_BS53_ENC4_cDNA650,/biosample-types/tissue_UBERON_0006483/,mid frontal cortex,barbara-wold:ENC4_cDNA650,/organisms/human/,/sources/rush-university/,john-stamatoyannopoulos:E4163791,barbara-wold,UM1HG009443
2,NaN,NaN,E4368365,90 or above,year,male,NINCDS PROB AD,24040,ENC4_cDNA651,human_brain_Rush ID_4368365_BS62_ENC4_cDNA651,/biosample-types/tissue_UBERON_0006483/,mid frontal cortex,barbara-wold:ENC4_cDNA651,/organisms/human/,/sources/rush-university/,john-stamatoyannopoulos:E4368365,barbara-wold,UM1HG009443
3,NaN,NaN,E4933693,90 or above,year,female,NINCDS PROB AD,24041,ENC4_cDNA652,human_brain_Rush ID_4933693_BS56_ENC4_cDNA652,/biosample-types/tissue_UBERON_0006483/,mid frontal cortex,barbara-wold:ENC4_cDNA652,/organisms/human/,/sources/rush-university/,john-stamatoyannopoulos:E4933693,barbara-wold,UM1HG009443
4,NaN,NaN,E5115600,88,year,female,NCI,24042,ENC4_cDNA653,human_brain_Rush ID_5115600_BS16_ENC4_cDNA653,/biosample-types/tissue_UBERON_0006483/,mid frontal cortex,barbara-wold:ENC4_cDNA653,/organisms/human/,/sources/rush-university/,john-stamatoyannopoulos:E5115600,barbara-wold,UM1HG009443
5,NaN,NaN,E5194210,79,year,female,NCI,24043,ENC4_cDNA654,human_brain_Rush ID_5194210_BS19_ENC4_cDNA654,/biosample-types/tissue_UBERON_0006483/,mid frontal cortex,barbara-wold:ENC4_cDNA654,/organisms/human/,/sources/rush-university/,john-stamatoyannopoulos:E5194210,barbara-wold,UM1HG009443
6,NaN,NaN,E6207143,90 or above,year,female,NINCDS PROB AD,24044,ENC4_cDNA655,human_brain_Rush ID_6207143_BS26_ENC4_cDNA655,/biosample-types/tissue_UBERON_0006483/,mid frontal cortex,barbara-wold:ENC4_cDNA655,/organisms/human/,/sources/rush-university/,john-stamatoyannopoulos:E6207143,barbara-wold,UM1HG009443
7,NaN,NaN,E6341028,90 or above,year,female,NINCDS PROB AD,24045,ENC4_cDNA656,human_brain_Rush ID_6341028_BS46_ENC4_cDNA656,/biosample-types/tissue_UBERON_0006483/,mid frontal cortex,barbara-wold:ENC4_cDNA656,/organisms/human/,/sources/rush-university/,john-stamatoyannopoulos:E6341028,barbara-wold,UM1HG009443
8,NaN,NaN,E7948794,90 or above,year,female,NINCDS PROB AD,24046,ENC4_cDNA657,human_brain_Rush ID_7948794_BS24_ENC4_cDNA657,/biosample-types/tissue_UBERON_0006483/,mid frontal cortex,barbara-wold:ENC4_cDNA657,/organisms/human/,/sources/rush-university/,john-stamatoyannopoulos:E7948794,barbara-wold,UM1HG009443
9,NaN,NaN,E8074960,89,year,female,NINCDS PROB AD,24047,ENC4_cDNA658,human_brain_Rush ID_8074960_BS43_ENC4_cDNA658,/biosample-types/tissue_UBERON_0006483/,mid frontal cortex,barbara-wold:ENC4_cDNA658,/organisms/human/,/sources/rush-university/,john-stamatoyannopoulos:E8074960,barbara-wold,UM1HG009443


In [12]:
#biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Lookup clinical data

In [13]:
print(spreadsheet_name)
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)

def add_if_needed(sheet, name, values):
    if name not in sheet.columns:
        sheet[name] = values
    return sheet

rush_ids = []
ages = []
age_units = []
sexes = []
clinical = []
for i, row in biosample.iterrows():
    donor = server.get_json(row["donor"])
    rush_ids.append(row["donor"].split(":")[1])
    ages.append(donor["age"]),
    age_units.append(donor["age_units"]),
    sexes.append(donor["sex"]),
    clinical.append(donor['submitter_comment'])
    
add_if_needed(biosample, "rush_id:skip", rush_ids)
add_if_needed(biosample, "age:skip", ages)
add_if_needed(biosample, "age_units:skip", age_units)
add_if_needed(biosample, "sex:skip", sexes)
add_if_needed(biosample, "clinical_status:skip", clinical)

biosample.to_excel("/dev/shm/biosample.xlsx")

/home/diane/woldlab/ENCODE/stranded-24038-24061-rush-resubmit.xlsx


# Register Biosamples

In [26]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)
created = server.post_sheet('/biosamples/', biosample, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

0


In [25]:
if created:
    biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Retrieve library starting amount

In [16]:
print(spreadsheet_name)
libraries = pandas.read_excel(spreadsheet_name, sheet_name='Library', header=0, engine=engine)

fragment_size = []
for i, row in libraries.iterrows():
    library_id = row["library_id:skip"]
    library_info = htsw.get_library(library_id)    
    if pandas.isnull(row["average_fragment_size:integer"]):
        fragment_size.append(library_info["insert_size"])
    else:
        assert library_info["insert_size"] == row["average_fragment_size:integer"], "{} {} {}!={}".format(i, library_id, library_info["insert_size"], row["average_fragment_size:integer"])
        fragment_size.append(row["average_fragment_size:integer"])
    
#fragment_size

/home/diane/woldlab/ENCODE/stranded-24038-24061-rush-resubmit.xlsx


# Register Libraries

In [30]:
print(spreadsheet_name)
libraries = pandas.read_excel(spreadsheet_name, sheet_name='Library', header=0, engine=engine)
created = server.post_sheet('/libraries/', 
                            libraries,
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

/home/diane/woldlab/ENCODE/stranded-24038-24061-rush-resubmit.xlsx
0


In [29]:
if created:
    libraries.to_excel('/dev/shm/libraries.xlsx', index=False)

# Register Experiments

In [33]:
print(server.server)
experiments = pandas.read_excel(spreadsheet_name, sheet_name='Experiment', header=0, engine=engine)
experiments = experiments[experiments['accession'] != 'barbara approval needed']
created = server.post_sheet('/experiments/', 
                            experiments, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

www.encodeproject.org
0


In [32]:
if created:
    experiments.to_excel('/dev/shm/experiments.xlsx', index=False)

# Register Replicates

In [37]:
print(server.server)
print(spreadsheet_name)
replicates = pandas.read_excel(spreadsheet_name, sheet_name='Replicate', header=0, engine=engine)
replicates = replicates[replicates['uuid'] != 'barbara approval needed']
created = server.post_sheet('/replicates/',
                            replicates, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

www.encodeproject.org
/home/diane/woldlab/ENCODE/stranded-24038-24061-rush-resubmit.xlsx
0


In [36]:
if created:
    replicates.to_excel('/dev/shm/replicates.xlsx', index=False)

# Check Files

In [23]:
files = pandas.read_excel(spreadsheet_name, sheet_name='File', header=0, engine=engine)
created = server.post_sheet('/files/', files, verbose=True, dry_run=True, validator=validator)
print(len(created))

24
